# Creating a playlist with my most-played songs this month, half year or year

In [125]:
# Variables
my_spotify_userid = 114770937
playlist_name = "Top songs this year"
playlist_description = "Automatically generated by Spotify Web API from my top 50 most played songs this year"
time_range = "long_term" #short_term is aobut 1 month, medium_term about 6 months and long_term about 1 year

In [126]:
import requests
from urllib.parse import urlencode
import base64
import webbrowser
import pandas as pd
import json

# Connecting to the Spotify API

In [127]:
with open("key.txt", "r") as file:
    secret_key = file.read()
client_id = "d41778f4ffda4294ae57e791c1516468"
client_secret = secret_key

## Authorization for playlist creation and modification

In [128]:
auth_headers_modify_playlist = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "playlist-modify-public"
}
webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers_create_playlist))

True

In [129]:
# [manual step] Copy this code from the url of the webpage that opens after the code above is executed
code = "fill in code here"
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}
token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}
r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
token_modify_playlist = r.json()["access_token"]

## Authorization for getting my most listened tracks

In [130]:
auth_headers_top_tracks = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-top-read"
}
webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers_top_tracks))

True

In [131]:
# [manual step] Copy this code from the url of the webpage that opens after the code above is executed
code = "fill in code here"
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}
token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}
r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
token_top_tracks = r.json()["access_token"]

# Create a new playlist

In [132]:
user_headers_modify_playlist = {
    "Authorization": "Bearer " + token_modify_playlist,
    "Content-Type": "application/json"
}

data = {
    "name": playlist_name,
    "description": playlist_description
}

In [133]:
response = requests.post(f"https://api.spotify.com/v1/users/{my_spotify_userid}/playlists", headers=user_headers_modify_playlist, data=json.dumps(data))

In [134]:
playlist_id = response.json()['id']

# Get my top tracks

In [135]:
user_headers_top_tracks = {
    "Authorization": "Bearer " + token_top_tracks,
    "Content-Type": "application/json"
}

In [136]:
top_tracks_response = requests.get(f"https://api.spotify.com/v1/me/top/tracks?time_range={time_range}&limit=50&offset=0", headers=user_headers_top_tracks)

# Add them to the playlist

In [137]:
track_uris = ["spotify:track:" + track_id for track_id in track_ids]

data = {
    "uris": track_uris,
    "position": 0
}

requests.post(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=user_headers_modify_playlist, data=json.dumps(data))

<Response [200]>